In [6]:
from tools import *

# NQM2024 contract
print("Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)")
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

ib.sleep(5)


def print_orderbook():
    print("NQ Order Book::")
    if ticker is not None and ticker.domBids is not None and ticker.domAsks is not None:
        for i in range(min(len(ticker.domBids), len(ticker.domAsks))):
            bid_size = ticker.domBids[i].size
            bid_price = ticker.domBids[i].price
            ask_price = ticker.domAsks[i].price
            ask_size = ticker.domAsks[i].size
            print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")
    print()


print_orderbook()


2024-05-24 10:09:33,164 ib_async.wrapper INFO Warning 2119, reqId -1: Market data farm is connecting:usfuture
2024-05-24 10:09:33,563 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfuture
2024-05-24 10:09:45,045 ib_async.wrapper INFO openOrder: Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=Order(orderId=379, clientId=6, permId=1735430611, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=18670.25, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=379, status='Submitted', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFill

Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)
NQ Order Book::
     3.0   18891.25 | 18891.5    1.0     
     9.0    18891.0 | 18891.75   7.0     
     6.0   18890.75 | 18892.0    7.0     
     8.0    18890.5 | 18892.25   8.0     
     9.0   18890.25 | 18892.5    6.0     



In [7]:
def parse_ibrecords(data_array):

    data_list = []

    for obj in data_array:
        data = {}

        if hasattr(obj, "contract"):
            util.logging.debug(obj.contract)
            contract = util.dataclassNonDefaults(obj.contract)
            data = {**data, **contract}

        if hasattr(obj, "order"):
            util.logging.debug(obj.order)
            order = util.dataclassNonDefaults(obj.order)
            order.pop("softDollarTier")
            data = {**data, **order}

        if hasattr(obj, "orderStatus"):
            util.logging.debug(obj.orderStatus)
            orderStatus = util.dataclassNonDefaults(obj.orderStatus)
            data = {**data, **orderStatus}

        if hasattr(obj, "fills"):
            util.logging.debug(obj.fills)
            fills = {"fills": obj.fills}
            data = {**data, **fills}

        if hasattr(obj, "log"):
            util.logging.debug(obj.log)
            logs = {"log": [util.dataclassAsDict(e) for e in obj.log]}
            data = {**data, **logs}

        if hasattr(obj, "advancedError"):
            util.logging.debug(obj.advancedError)
            advancedError = {"advancedError": obj.advancedError}
            data = {**data, **advancedError}

        if type(obj) == Order:
            util.logging.debug(obj)
            order = util.dataclassNonDefaults(obj)
            order.pop("softDollarTier")
            data = {**data, **order}

        data_list.append(data)

    return data_list


def print_ibrecords_table(
    data_array,
    cols,
):
    df = pd.DataFrame(data_array)
    df = df[cols]
    display(df)


# # openTrades = parse_ibrecords(ib.openTrades())
accountSummary = ib.accountSummary()
accountValues = ib.accountValues()

# openTrades = parse_ibrecords(ib.openTrades())
# util.df([d for d in openTrades])

# openOrders = parse_ibrecords(ib.openOrders())
# util.df([d for d in openOrders])

# trades = parse_ibrecords(ib.trades())
# util.df([d for d in trades])

# orders = parse_ibrecords(ib.orders())
# util.df([d for d in orders])


In [8]:
ib.reqAllOpenOrders()


ORDER_COLS = [
    "localSymbol",
    "permId",
    "status",
    "orderType",
    "action",
    "lmtPrice",
    "remaining",
]

TRADE_COLS = [
    "clientId",
    "localSymbol",
    "status",
    "orderType",
    "action",
    "lmtPrice",
    "remaining",
    "filledQuantity",
    "fills",
]

OPEN_TRADE_COLS = [
    "permId",
    "orderId",
    "symbol",
    "lastTradeDateOrContractMonth",
    "orderType",
    "action",
    "lmtPrice",
    "totalQuantity",
    "remaining",
    "fills",
    "log",
]

# trades = parse_ibrecords(ib.trades())
# trades_df = util.df([d for d in trades])
# print_ibrecords_table(trades_df, cols=TRADE_COLS)

# open_trades = parse_ibrecords(ib.openTrades())
# open_trades_df = util.df([d for d in open_trades])
# open_trades_df.columns

# print_ibrecords_table(open_trades_df, cols=OPEN_TRADE_COLS)


In [ ]:
trades_df.loc[
    :,
    [
        "clientId",
        "localSymbol",
        "status",
        "orderType",
        "action",
        "lmtPrice",
        "remaining",
        "filledQuantity",
        "fills",
    ],
]


In [ ]:
def print_openTrades():
    print(":: openTrades ::")
    util.df([t.order for t in ib.openTrades()]).sort_values("permId").loc[
        :,
        [
            "orderId",
            "clientId",
            "permId",
            "action",
            "totalQuantity",
            "orderType",
            "lmtPrice",
            "auxPrice",
            "tif",
        ],
    ]


print_openTrades()


In [ ]:
util.df(ib.positions()).sort_values("account", ascending=False)


In [18]:
def run_strategy(strategy_details, open_permId, close_permId, cancel_permids=[]):
    open_trade = None
    close_trade = None
    open_order = None
    close_order = None
    open_order_ts = None

    push_notifications(
        f"-------- {strategy_details} / {open_permId} / {close_permId} / {cancel_permids}"
    )

    if open_permId is not None:
        orders = [t.order for t in ib.reqAllOpenOrders()]
        for o in orders:
            if o.permId == open_permId:
                open_order = o

    if open_order is not None:
        push_notifications(f"OPEN ORDER:: {open_order}")

    if open_trade is None:
        trades = ib.trades()
        for o in trades:
            if o.orderStatus.permId == open_permId:
                open_trade = o

    # push_msg(f"OPEN TRADE:: {open_trade.order}")

    if close_permId is not None:
        for o in orders:
            if o.permId == close_permId:
                close_order = o

    if close_order is not None:
        push_notifications(f"CLOSE ORDER:: {close_order}")

    if close_trade is None:
        trades = ib.trades()
        for o in trades:
            if o.orderStatus.permId == close_permId:
                close_trade = o

    # print(f"CLOSE TRADE:: {close_trade}")

    for permId in cancel_permids:
        cancelled_order = None
        for o in orders:
            if o.permId == permId:
                print("Cancelling order.permId {o.permId}")
                cancelled_order = ib.cancelOrder(o)
                ib.sleep(2)
                if cancelled_order.orderStatus.status == "Cancelled":
                    print(f"Order {o.permId} has been cancelled")
                else:
                    raise Exception("Unable to cancel order {o.permId}")

        if cancelled_order is None:
            raise Exception(":: ORDER TO CANCEL NOT FOUND ::  {permId}")

    pause = False
    ib.sleep(1)

    while True:
        clear_output(wait=True)

        # first order of the strategy
        if open_trade is None and close_trade is None:
            action = strategy_details["open_action"]
            qty = strategy_details["open_qty"]

            if ticker is None or len(ticker.domBids) == 0 or len(ticker.domAsks) == 0:
                print("************ Issue getting orderbok *************** ")
                ib.sleep(1)
                continue

            if strategy_details["open_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["open_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["open_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            elif strategy_details["open_ref"] == "last":
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["open_ticks"] * strategy_details["tick_increment"]
            )
            print(
                f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}"
            )

            if strategy_details["open_type"] == "LIMIT":
                open_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            open_trade = ib.placeOrder(contract, open_order)
            trade = open_trade

            push_notifications(
                f"OPEN ORDER PLACED :: #{trade.order.permId} {trade.orderStatus.status} {trade.contract.symbol} {trade.order.action} {trade.orderStatus.filled}/{trade.orderStatus.remaining} @ {trade.order.lmtPrice}"
            )

            open_order_ts = datetime.datetime.now()

        print_order(open_trade)
        print(
            "--------------------------------------------------------------------------------------"
        )

        if open_trade is not None:
            if open_trade.orderStatus.status == "Submitted" and close_trade is None:
                print(
                    f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})"
                )

                if (
                    open_order_ts is None
                    or datetime.datetime.now() - open_order_ts
                    > datetime.timedelta(seconds=strategy_details["pause_seconds"])
                ):
                    print("Cancelling order due to timeout:")
                    ib.cancelOrder(open_trade.order)

            if open_trade.orderStatus.status == "Filled" and close_trade is None:
                print(
                    "Filled on open_trade {open_trade.orderStatus.status} {open_trade.orderStatus.avgFillPrice} {open_trade.orderStatus.status}"
                )

                action = strategy_details["close_action"]
                qty = strategy_details["close_qty"]

                if strategy_details["close_ref"] == "open_price_fill":
                    price_ref = open_trade.orderStatus.avgFillPrice
                if strategy_details["close_ref"] == "bid":
                    price_ref = ticker.domBids[0].price
                elif strategy_details["close_ref"] == "ask":
                    price_ref = ticker.domAsks[0].price
                elif strategy_details["close_ref"] == "mid":
                    price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
                elif strategy_details["close_ref"] == "last":
                    raise Exception("Not implemented")

                lmtPrice = (
                    price_ref
                    + strategy_details["close_ticks"]
                    * strategy_details["tick_increment"]
                )

                if strategy_details["close_type"] == "LIMIT":
                    close_order = LimitOrder(
                        action=action,
                        totalQuantity=qty,
                        lmtPrice=lmtPrice,
                        account="U10394496",
                    )
                else:
                    raise Exception("Not implemented")

                close_trade = ib.placeOrder(contract, close_order)

                # chime.info()
                push_notifications(msg=f"OPEN ORDER FILLED:: {open_trade.order}")
                push_notifications(msg=f"CLOSE ORDER PLACED:: {close_trade.order}")

            elif (
                open_trade.orderStatus.status == "Inactive"
                or open_trade.orderStatus.status == "Cancelled"
            ) and close_trade is None:
                # push_notifications(f"OPEN TRADE CANCELLED:: {open_trade.order}")
                open_trade = None

        print_order(close_trade)
        print(
            "--------------------------------------------------------------------------------------"
        )

        if close_trade is not None:
            if close_trade.orderStatus.status == "Filled":
                print(
                    "Close trade filled @ {}".format(
                        close_trade.orderStatus.avgFillPrice
                    )
                )
                os.system("say beep")
                chime_success()
                push_notifications(f"CLOSE TRADE FILLED:: {close_trade.order}")
                open_trade = None
                close_trade = None
                pause = True

                print(
                    "--------------------------------------------------------------------------------------"
                )

        if (
            ticker is not None
            and ticker.domBids is not None
            and ticker.domAsks is not None
        ):
            max_length = max(len(ticker.domBids), len(ticker.domAsks))
            for i in range(max_length):
                bid_size = ticker.domBids[i].size if i < len(ticker.domBids) else "-"
                bid_price = ticker.domBids[i].price if i < len(ticker.domBids) else "-"
                ask_price = ticker.domAsks[i].price if i < len(ticker.domAsks) else "-"
                ask_size = ticker.domAsks[i].size if i < len(ticker.domAsks) else "-"
                print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")

        print(
            "--------------------------------------------------------------------------------------"
        )
        future = [pos for pos in ib.positions() if pos.contract.secType == "FUT"]
        for f in future:
            print(
                f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)} ({len(ib.executions())} executions)"
            )
        print(
            "--------------------------------------------------------------------------------------"
        )

        print(
            util.df(ib.openOrders())
            .sort_values("lmtPrice", ascending=False)
            .loc[
                :,
                [
                    "orderId",
                    "permId",
                    "action",
                    "totalQuantity",
                    "orderType",
                    "lmtPrice",
                    "tif",
                ],
            ]
        )

        # openOrders = parse_ibrecords(ib.openOrders())
        # print_ibrecords_table(
        #     openOrders,
        #     cols=[
        #         "localSymbol",
        #         "permId",
        #         "status",
        #         "orderType",
        #         "action",
        #         "lmtPrice",
        #         "remaining",
        #     ],
        # )

        print()
        print("------- U10394496 ----------")

        for item in ib.accountSummary(account="U10394496"):
            if item.tag in [
                "Account",
                "Cushion",
                "NetLiquidation",
                "TotalCashValue",
                "LookAheadExcessLiquidity",
            ]:
                print(item.tag, item.value)

        print("------- U2340948 ----------")

        for item in ib.accountSummary(account="U2340948"):
            if item.tag in [
                "Account",
                "Cushion",
                "NetLiquidation",
                "FullExcessLiquidity",
                "LookAheadExcessLiquidity",
            ]:
                print(item.tag, item.value)

        if pause:
            open_orders = ib.reqAllOpenOrders()
            executions = ib.reqExecutions()

            ib.sleep(strategy_details["pause_seconds"])
            pause = False
        else:
            ib.sleep(1)


sell_scalp = {
    "strategy": "SELL TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10,
    "pause_seconds": 90,
}

buy_scalp = {
    "strategy": "BUY TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "ask",
    "open_ticks": -5,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10,
    "pause_seconds": 70,
}


run_strategy(
    strategy_details=buy_scalp,
    open_permId=527389862,
    close_permId=527389863,
    cancel_permids=[],
)


symbol	permId		status		action	filled	remaining	lmtPrice
NQ	527389862	Filled	BUY	1.0	0.0		18922.5	
--------------------------------------------------------------------------------------
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	527389863	Submitted	SELL	0.0	1.0		18925.0	
--------------------------------------------------------------------------------------
     3.0    18862.0 | 18862.5    7.0     
     9.0   18861.75 | 18862.75   7.0     
     7.0    18861.5 | 18863.0    8.0     
    10.0   18861.25 | 18863.25   8.0     
    11.0    18861.0 | 18863.5    10.0    
--------------------------------------------------------------------------------------
NQ -2.0 @ 18920.816666667502 (58 executions)
--------------------------------------------------------------------------------------
   orderId      permId action  totalQuantity orderType  lmtPrice  tif
3      416   527389863   SELL            1.0       LMT  18925.00     
2      271   196158554   SELL            1.0       LMT  1

KeyboardInterrupt: 

In [ ]:
trades = ib.trades()
trades[-1]


In [ ]:
trades = parse_ibrecords(ib.trades())
submittedTrades = [o for o in trades if o["status"] == "Submitted"]

print_ibrecords_table(
    submittedTrades,
    ["orderId", "permId", "status", "orderType", "action", "lmtPrice", "remaining"],
)


In [ ]:
def parse_ibrecords(data_array):

    data_list = []

    for obj in data_array:
        data = {}

        if hasattr(obj, "contract"):
            util.logging.debug(obj.contract)
            contract = util.dataclassNonDefaults(obj.contract)
            data = {**data, **contract}

        if hasattr(obj, "order"):
            util.logging.debug(obj.order)
            order = util.dataclassNonDefaults(obj.order)
            order.pop("softDollarTier")
            data = {**data, **order}

        if hasattr(obj, "orderStatus"):
            util.logging.debug(obj.orderStatus)
            orderStatus = util.dataclassNonDefaults(obj.orderStatus)
            data = {**data, **orderStatus}

        if hasattr(obj, "fills"):
            util.logging.debug(obj.fills)
            fills = {"fills": obj.fills}
            data = {**data, **fills}

        if hasattr(obj, "log"):
            util.logging.debug(obj.log)
            logs = {"log": [util.dataclassAsDict(e) for e in obj.log]}
            data = {**data, **logs}

        if hasattr(obj, "advancedError"):
            util.logging.debug(obj.advancedError)
            advancedError = {"advancedError": obj.advancedError}
            data = {**data, **advancedError}

        if type(obj) == Order:
            util.logging.debug(obj)
            order = util.dataclassNonDefaults(obj)
            order.pop("softDollarTier")
            data = {**data, **order}

        data_list.append(data)

    return data_list


def print_ibrecords_table(
    data_array,
    cols=[
        "localSymbol",
        "permId",
        "status",
        "orderType",
        "action",
        "lmtPrice",
        "remaining",
    ],
):
    df = pd.DataFrame(data_array)
    df = df[cols]
    print(df)


# openTrades = parse_ibrecords(ib.openTrades())
accountSummary = ib.accountSummary()
accountValues = ib.accountValues()

# openTrades = parse_ibrecords(ib.openTrades())
# # util.df([d for d in openTrades])

# print_ibrecords_table(openTrades)

# openOrders = parse_ibrecords(ib.openOrders())
# util.df([d for d in openOrders])

# trades = parse_ibrecords(ib.trades())
# util.df([d for d in trades])

# orders = parse_ibrecords(ib.orders())
# util.df([d for d in orders])


openOrders = parse_ibrecords(ib.openOrders())
openOrders


In [ ]:
print_ibrecords_table(
    openOrders,
    cols=[
        "localSymbol",
        "permId",
        "status",
        "orderType",
        "action",
        "lmtPrice",
        "remaining",
    ],
)


In [ ]:
ord = ib.orders()

[print(o) for o in ord if o.permId == 1394515162]


In [ ]:
util.df(ib.orders())


In [ ]:
util.df(ib.openOrders())


In [ ]:
from datetime import datetime

input_string = "20240930 16:00:00 Central Standard Time"
date_format = "%Y%m%d %H:%M:%S %Z"

try:
    datetime_object = datetime.strptime(input_string, date_format)
    print(datetime_object)
except ValueError:
    print("Incorrect date format.")


In [ ]:
util.df([t for t in ib.reqAllOpenOrders()])


In [ ]:
clear_output()
o = ib.orders()[0]
o = util.dataclassNonDefaults(o)
o.pop("softDollarTier", None)
o.pop("shareholder", None)
o.pop("autoCancelDate", None)
o.pop("auxPrice", None)

data, count = supabase.table("orders").upsert(o).execute()

util.tree(o)


In [ ]:
o["autoCancelDate"]


In [ ]:
od = util.dataclassAsDict(o)
print(od)


In [ ]:
clear_output()
o = ib.orders()[0]
od = util.dataclassAsDict(o)

data, count = supabase.table("orders").upsert(od).execute()


In [ ]:
clear_output()
util.dataclassRepr(ib.trades()[0])

# util.dataclassAsDict(executions[0])


In [ ]:
# Get Account details
details = ib.accountSummary()
details_df = util.df(details)

# Tag Values
# tags = details_df['tag'].values.tolist()
# print(tags)

# Set tag values as index
details_df = details_df.set_index("tag")
net_liquidation_value = details_df.loc["NetLiquidation", "value"]
details_df


In [ ]:
util.tree(ib.trades()[0])


In [ ]:
util.tree(ib.openTrades()[0])


In [ ]:
util.tree(ib.openTrades()[1])


In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )


In [ ]:
trades_df = util.df(ib.trades())
trades_df


In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("trades.db")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create the "trades" table if it doesn't exist
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS trades (
        contract TEXT,
        order TEXT,
        orderStatus TEXT,
        fills TEXT,
        log TEXT,
        advancedError TEXT
    )
"""
)

# Convert the DataFrame to a list of tuples
trades_data = trades_df.to_records(index=False).tolist()

# Insert the data into the "trades" table
cursor.executemany(
    """
    INSERT INTO trades (contract, order, orderStatus, fills, log, advancedError)
    VALUES (?, ?, ?, ?, ?, ?)
""",
    trades_data,
)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')
